In [1]:
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Load features
train_manual = np.load("featuresets/train_manual_features.npy")
test_manual = np.load("featuresets/test_manual_features.npy")

train_mobilenet = np.load("featuresets/train_mobilenet_features.npy")
test_mobilenet = np.load("featuresets/test_mobilenet_features.npy")

train_densenet = np.load("featuresets/train_densenet_features.npy")
test_densenet = np.load("featuresets/test_densenet_features.npy")

train_labels = np.load("featuresets/train_manual_labels.npy")  # All labels are aligned
test_labels = np.load("featuresets/test_manual_labels.npy")

# Check dimensions
print("Manual:", train_manual.shape)
print("MobileNet:", train_mobilenet.shape)
print("DenseNet:", train_densenet.shape)
print("Labels:", train_labels.shape)

# Combine features
X_train = np.hstack((train_manual, train_mobilenet, train_densenet))
X_test = np.hstack((test_manual, test_mobilenet, test_densenet))

print("Combined train shape:", X_train.shape)
print("Combined test shape:", X_test.shape)


Manual: (8400, 14)
MobileNet: (8400, 1280)
DenseNet: (8400, 1024)
Labels: (8400,)
Combined train shape: (8400, 2318)
Combined test shape: (1400, 2318)


In [3]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# 🔁 Load combined feature arrays and labels (already loaded if you're in Colab)
# X_train, X_test, train_labels, test_labels already defined
# If not, reload them before this block.

# 🎯 Remove class 6 from training and testing sets
class_to_remove = 6

train_mask = train_labels != class_to_remove
test_mask = test_labels != class_to_remove

X_train_filtered = X_train[train_mask]
X_test_filtered = X_test[test_mask]
y_train_filtered = train_labels[train_mask]
y_test_filtered = test_labels[test_mask]

# 🔄 Train SVM
svm = SVC(probability=True, kernel='rbf', C=10, gamma='scale', random_state=42)
svm.fit(X_train_filtered, y_train_filtered)
y_pred = svm.predict(X_test_filtered)

# 📊 Evaluate
print("\n🧼 Removed Class 6")
print(classification_report(y_test_filtered, y_pred))
print("Filtered SVM Accuracy:", accuracy_score(y_test_filtered, y_pred))



🧼 Removed Class 6
              precision    recall  f1-score   support

           0       0.75      0.75      0.75       200
           1       0.89      0.92      0.90       200
           2       0.77      0.78      0.78       200
           3       0.82      0.82      0.82       200
           4       0.90      0.88      0.89       200
           5       0.80      0.79      0.80       200

    accuracy                           0.82      1200
   macro avg       0.82      0.82      0.82      1200
weighted avg       0.82      0.82      0.82      1200

Filtered SVM Accuracy: 0.8225


In [4]:
rf = RandomForestClassifier(n_estimators=150, random_state=42)
rf.fit(X_train_filtered, y_train_filtered)
rf_pred = rf.predict(X_test_filtered)

print("\n🌲 Random Forest")
print(classification_report(y_test_filtered, rf_pred))
print("Random Forest Accuracy:", accuracy_score(y_test_filtered, rf_pred))


🌲 Random Forest
              precision    recall  f1-score   support

           0       0.76      0.64      0.69       200
           1       0.83      0.91      0.87       200
           2       0.73      0.82      0.77       200
           3       0.82      0.77      0.79       200
           4       0.89      0.88      0.88       200
           5       0.78      0.79      0.78       200

    accuracy                           0.80      1200
   macro avg       0.80      0.80      0.80      1200
weighted avg       0.80      0.80      0.80      1200

Random Forest Accuracy: 0.8008333333333333


In [5]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb.fit(X_train_filtered, y_train_filtered)
xgb_pred = xgb.predict(X_test_filtered)

print("\n⚡ XGBoost")
print(classification_report(y_test_filtered, xgb_pred))
print("XGBoost Accuracy:", accuracy_score(y_test_filtered, xgb_pred))

D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:24:46] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



⚡ XGBoost
              precision    recall  f1-score   support

           0       0.75      0.70      0.73       200
           1       0.87      0.94      0.90       200
           2       0.75      0.78      0.76       200
           3       0.87      0.77      0.81       200
           4       0.88      0.91      0.89       200
           5       0.79      0.80      0.79       200

    accuracy                           0.82      1200
   macro avg       0.82      0.82      0.82      1200
weighted avg       0.82      0.82      0.82      1200

XGBoost Accuracy: 0.8166666666666667


In [6]:
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_filtered, y_train_filtered)
lr_pred = lr.predict(X_test_filtered)

print("\n📊 Logistic Regression")
print(classification_report(y_test_filtered, lr_pred))
print("Logistic Regression Accuracy:", accuracy_score(y_test_filtered, lr_pred))


📊 Logistic Regression
              precision    recall  f1-score   support

           0       0.66      0.62      0.64       200
           1       0.85      0.91      0.87       200
           2       0.68      0.71      0.69       200
           3       0.81      0.76      0.78       200
           4       0.88      0.88      0.88       200
           5       0.69      0.69      0.69       200

    accuracy                           0.76      1200
   macro avg       0.76      0.76      0.76      1200
weighted avg       0.76      0.76      0.76      1200

Logistic Regression Accuracy: 0.7616666666666667
